In [1]:
import pandas as pd
import numpy as np
from ta import add_all_ta_features

%run C:\Projects\Stock_price_prediction\Experiments_YHdata\data_processing_functions.ipynb
%run C:\Projects\Stock_price_prediction\Experiments_YHdata\feature_engineering.ipynb

# 2019-2024 data

In [2]:
# For importing pre-processed data and further process here
data_path = 'C:/Projects/Stock_price_prediction/Experiments_YHdata/YH_raw_20180103-20241206.csv'
TBill_path = 'C:/Projects/Stock_price_prediction/Experiments_YHdata/1yearTBill_20050103_20241205.csv'

In [3]:
data = pd.read_csv(data_path)
data['datadate'] = pd.to_datetime(data['datadate'])
data

,datadate,tic,prcod,prchd,prcld,prccd,cshtrd,sector,dol_vol
0,2018-01-03,A,67.620003,69.489998,67.599998,69.320000,1698900.0,Healthcare,1.148796e+08
1,2018-01-04,A,69.540001,69.820000,68.779999,68.800003,2230700.0,Healthcare,1.551229e+08
2,2018-01-05,A,68.730003,70.099998,68.730003,69.900002,1632500.0,Healthcare,1.122017e+08
3,2018-01-08,A,69.730003,70.330002,69.550003,70.050003,1613400.0,Healthcare,1.125024e+08
4,2018-01-09,A,70.680000,72.330002,70.169998,71.769997,2666100.0,Healthcare,1.884399e+08
...,...,...,...,...,...,...,...,...,...
3750931,2024-11-29,ZWS,40.000000,40.060001,39.610001,39.820000,429100.0,Industrials,1.716400e+07
3750932,2024-12-02,ZWS,40.090000,40.400002,39.689999,40.279999,1118400.0,Industrials,4.483666e+07
3750933,2024-12-03,ZWS,40.299999,40.740002,39.840000,40.320000,663000.0,Industrials,2.671890e+07
3750934,2024-12-04,ZWS,40.450001,40.840000,40.080002,40.770000,851300.0,Industrials,3.443509e+07


In [4]:
data = compute_ret(data, TBill_path)
data = remove_dead_stocks(data)

data = assign_class_labels(data, 'fixed_size')
# Make sector column
data, num_of_tokens, num_to_sector_dict = make_sector_column(data)
# Create dictionary that associate each ticker with a numerical label and vice versa for easier reference
num_to_tic_dict, tic_to_num_dict = num_tic_dicts(data)

print(f'Confirm that data has no NAs: {~data.isna().any().any()}')

EXPRQ is dead after 2024-08-14T00:00:00.000000000
IDEX is dead after 2024-08-13T00:00:00.000000000
SAVE is dead after 2024-11-21T00:00:00.000000000
SRNE is dead after 2023-09-28T00:00:00.000000000
VTNR is dead after 2024-10-07T00:00:00.000000000
ZOM is dead after 2020-09-24T00:00:00.000000000
Confirm that data has no NAs: True


C:\Users\rguo\AppData\Local\Temp\ipykernel_35764\3379698402.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['sector'] = data['sector'].replace(renumber_sectors)


In [5]:
factors = ['cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol']

data, factors = feature_engineer(data, factors)
print(f'Confirm that feature engineering did not create NaNs: {~data.isna().any().any()}')
print(f'All factors: {factors}')

assert data.shape[0] == data.datadate.nunique() * data.tic.nunique()

data

Confirm data has no NAs: True
Confirm that feature engineering did not create NaNs: True
All factors: ['cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_10day', 'MA_50day', 'open/MA10', 'open/MA50', 'STD_10day', 'H-L', 'RSI', 'MACD', 'MACD_Signal_Line']


,datadate,tic,prcod,prchd,prcld,prccd,cshtrd,sector,dol_vol,ret_d,...,Mom_5day,MA_10day,MA_50day,open/MA10,open/MA50,STD_10day,H-L,RSI,MACD,MACD_Signal_Line
0,2018-01-03,A,67.620003,69.489998,67.599998,69.320000,1698900.0,0,1.148796e+08,-0.011648,...,0.000000,67.620003,67.620003,1.000000,1.000000,0.000000,1.889999,53.120091,0.000000,0.000000
1,2018-01-04,A,69.540001,69.820000,68.779999,68.800003,2230700.0,0,1.551229e+08,0.014550,...,0.000000,68.580002,68.580002,1.013998,1.013998,1.357644,1.040001,100.000000,0.153162,0.030632
2,2018-01-05,A,68.730003,70.099998,68.730003,69.900002,1632500.0,0,1.122017e+08,0.013624,...,0.000000,68.630002,68.630002,1.001457,1.001457,0.963898,1.369995,70.329713,0.206801,0.065866
3,2018-01-08,A,69.730003,70.330002,69.550003,70.050003,1613400.0,0,1.125024e+08,0.010894,...,0.000000,68.905003,68.905003,1.011973,1.011973,0.960156,0.779999,78.284223,0.326240,0.117941
4,2018-01-09,A,70.680000,72.330002,70.169998,71.769997,2666100.0,0,1.884399e+08,-0.007418,...,0.000000,69.260002,69.260002,1.020502,1.020502,1.149586,2.160004,82.692333,0.491884,0.192729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750931,2024-11-29,ZWS,40.000000,40.060001,39.610001,39.820000,429100.0,2,1.716400e+07,0.005238,...,0.023018,39.422000,37.296400,1.014662,1.072490,0.558605,0.450001,58.807603,0.829485,0.856930
3750932,2024-12-02,ZWS,40.090000,40.400002,39.689999,40.279999,1118400.0,2,4.483666e+07,0.003722,...,0.011863,39.548000,37.401800,1.013705,1.071874,0.552304,0.710003,58.126721,0.818192,0.849182
3750933,2024-12-03,ZWS,40.299999,40.740002,39.840000,40.320000,663000.0,2,2.671890e+07,0.007911,...,0.003486,39.702999,37.514600,1.015037,1.074248,0.520022,0.900002,54.216849,0.816772,0.842700
3750934,2024-12-04,ZWS,40.450001,40.840000,40.080002,40.770000,851300.0,2,3.443509e+07,0.000000,...,0.012769,39.866999,37.637200,1.014624,1.074735,0.462531,0.759998,61.320773,0.818318,0.837824


In [6]:
# Add other technical features from the TA library
data = all_features_ta(data)
print(f'Confirm that data has no NaNs: {~data.isna().any().any()}')
data

C:\Users\rguo\anaconda3\envs\tf-gpu-clone\lib\site-packages\ta\volume.py:335: RuntimeWarning: overflow encountered in scalar multiply
  self._nvi.iloc[i] = self._nvi.iloc[i - 1] * (1.0 + price_change.iloc[i])
C:\Users\rguo\anaconda3\envs\tf-gpu-clone\lib\site-packages\ta\wrapper.py:496: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{colprefix}momentum_pvo_signal"] = indicator_pvo.pvo_signal()
C:\Users\rguo\anaconda3\envs\tf-gpu-clone\lib\site-packages\ta\wrapper.py:497: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{co

Confirm that data has no NaNs: True


,datadate,tic,prcod,prchd,prcld,prccd,cshtrd,sector,dol_vol,ret_d,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2018-01-03,A,67.620003,69.489998,67.599998,69.320000,1698900.0,0,1.148796e+08,-0.011648,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.620003,0.000000,0.000000,0.000000
1,2018-01-04,A,69.540001,69.820000,68.779999,68.800003,2230700.0,0,1.551229e+08,0.014550,...,0.226029,0.045206,0.180823,2.440486,0.488097,1.952389,68.536065,2.839394,2.799830,2.839394
2,2018-01-05,A,68.730003,70.099998,68.730003,69.900002,1632500.0,0,1.122017e+08,0.013624,...,0.304863,0.097137,0.207726,1.586687,0.707815,0.878871,68.617025,-1.164794,-1.171631,1.641527
3,2018-01-08,A,69.730003,70.330002,69.550003,70.050003,1613400.0,0,1.125024e+08,0.010894,...,0.479946,0.173699,0.306247,0.807009,0.727654,0.079355,69.088867,1.454969,1.444486,3.120379
4,2018-01-09,A,70.680000,72.330002,70.169998,71.769997,2666100.0,0,1.884399e+08,-0.007418,...,0.721504,0.283260,0.438244,4.860498,1.554223,3.306276,69.765581,1.362393,1.353196,4.525285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750931,2024-11-29,ZWS,40.000000,40.060001,39.610001,39.820000,429100.0,2,1.716400e+07,0.005238,...,2.147153,2.245926,-0.098773,-17.570550,-14.880085,-2.690465,38.643000,0.225508,0.225254,-40.845906
3750932,2024-12-02,ZWS,40.090000,40.400002,39.689999,40.279999,1118400.0,2,4.483666e+07,0.003722,...,2.112016,2.219144,-0.107128,-13.467597,-14.597588,1.129990,38.948755,0.225000,0.224748,-40.712809
3750933,2024-12-03,ZWS,40.299999,40.740002,39.840000,40.320000,663000.0,2,2.671890e+07,0.007911,...,2.102081,2.195731,-0.093650,-14.310676,-14.540205,0.229530,39.313851,0.523819,0.522452,-40.402251
3750934,2024-12-04,ZWS,40.450001,40.840000,40.080002,40.770000,851300.0,2,3.443509e+07,0.000000,...,2.099676,2.176520,-0.076844,-13.082938,-14.248752,1.165814,39.628169,0.372212,0.371521,-40.180421


In [7]:
TA_factors = [# Momentum indicators
              'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr',
              # Volume indicators
              'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt',
              # Volatility indicators
              'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility_ui',
              # Trend indicators
              'trend_adx', 'trend_aroon_up', 'trend_aroon_down', 'trend_ichimoku_a',
              # Other indicators
              'others_dr'
]

factors.extend(TA_factors)
print(factors)

data = data[['datadate', 'tic'] + factors + ['ret_d', 'TBill1y', 'rel_ret_d', 'DistinctRank', 'rank', 'sector']]
data

['cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_10day', 'MA_50day', 'open/MA10', 'open/MA50', 'STD_10day', 'H-L', 'RSI', 'MACD', 'MACD_Signal_Line', 'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr', 'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt', 'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility_ui', 'trend_adx', 'trend_aroon_up', 'trend_aroon_down', 'trend_ichimoku_a', 'others_dr']


,datadate,tic,cshtrd,prccd,prchd,prcld,prcod,dol_vol,Mom_2day,Mom_3day,...,trend_aroon_up,trend_aroon_down,trend_ichimoku_a,others_dr,ret_d,TBill1y,rel_ret_d,DistinctRank,rank,sector
0,2018-01-03,A,1698900.0,69.320000,69.489998,67.599998,67.620003,1.148796e+08,0.000000,0.000000,...,0.0,0.0,68.544998,0.000000,-0.011648,0.000071,-0.012974,310.0,-2,0
1,2018-01-04,A,2230700.0,68.800003,69.820000,68.779999,69.540001,1.551229e+08,0.000000,0.000000,...,4.0,0.0,68.709999,2.839394,0.014550,0.000072,0.012388,1886.0,2,0
2,2018-01-05,A,1632500.0,69.900002,70.099998,68.730003,68.730003,1.122017e+08,0.016415,0.000000,...,8.0,0.0,68.849998,-1.164794,0.013624,0.000071,0.010738,1775.0,2,0
3,2018-01-08,A,1613400.0,70.050003,70.330002,69.550003,69.730003,1.125024e+08,0.002732,0.031204,...,12.0,0.0,68.965000,1.454969,0.010894,0.000070,0.013818,1889.0,2,0
4,2018-01-09,A,2666100.0,71.769997,72.330002,70.169998,70.680000,1.884399e+08,0.028372,0.016393,...,16.0,0.0,69.965000,1.362393,-0.007418,0.000070,-0.010233,432.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750931,2024-11-29,ZWS,429100.0,39.820000,40.060001,39.610001,40.000000,1.716400e+07,0.001502,-0.003984,...,36.0,12.0,38.792500,0.225508,0.005238,0.000167,0.004944,1514.0,1,2
3750932,2024-12-02,ZWS,1118400.0,40.279999,40.400002,39.689999,40.090000,4.483666e+07,0.004510,0.003756,...,32.0,8.0,38.792500,0.225000,0.003722,0.000167,0.005939,1605.0,1,2
3750933,2024-12-03,ZWS,663000.0,40.320000,40.740002,39.840000,40.299999,2.671890e+07,0.007500,0.009772,...,100.0,4.0,38.867501,0.523819,0.007911,0.000166,-0.004699,1597.0,1,2
3750934,2024-12-04,ZWS,851300.0,40.770000,40.840000,40.080002,40.450001,3.443509e+07,0.008980,0.011250,...,100.0,0.0,38.927501,0.372212,0.000000,0.000164,0.000000,2152.0,2,2


In [8]:
data.to_csv('YH_processed_20180103-20241206.csv', index=False)

In [9]:
data['ret_d'].mean()

0.0005902299226753164

In [10]:
data['ret_d'].std()

0.1321656863815801

In [11]:
data['ret_d'].max()

248.9999980597741

In [12]:
data['ret_d'].min()

-0.9971428569179462

In [13]:
data[data['ret_d'] < -0.5].sort_values(by=['ret_d'])

,datadate,tic,cshtrd,prccd,prchd,prcld,prcod,dol_vol,Mom_2day,Mom_3day,...,trend_aroon_up,trend_aroon_down,trend_ichimoku_a,others_dr,ret_d,TBill1y,rel_ret_d,DistinctRank,rank,sector
1603477,2024-08-09,IDEX,54155.0,0.330000,0.500000,0.210000,0.500000,2.707750e+04,0.086957,0.515151,...,8.0,20.0,0.509975,21.951221,-0.997143,0.000175,-0.999587,1.0,-2,2
1117181,2024-08-12,EXPRQ,51775.0,0.320000,0.380000,0.312700,0.360000,1.863900e+04,-0.307692,-0.027027,...,0.0,44.0,0.440675,-18.181814,-0.964516,0.000174,-1.091409,1.0,-2,4
2794017,2024-11-19,SAVE,68138586.0,0.150100,0.490000,0.130000,0.257500,1.754569e+07,-0.804924,-0.804924,...,76.0,100.0,1.885000,-76.157409,-0.952381,0.000169,-0.959320,1.0,-2,2
583648,2023-11-29,CENN,118170.0,15.920000,21.000000,15.000000,21.000000,2.481570e+06,-0.116533,-0.085764,...,64.0,24.0,20.247500,-3.536980,-0.893316,0.000198,-0.892714,1.0,-2,4
3104304,2018-02-02,SVXY,13624650.0,211.199997,236.179993,206.220001,236.000000,3.215417e+09,0.000424,0.025106,...,40.0,12.0,189.610002,0.742767,-0.883000,0.000074,-0.844807,1.0,-2,6
2792276,2024-11-21,SAVA,4650600.0,32.180000,33.980000,26.030001,26.129999,1.215202e+08,-0.074389,-0.046350,...,100.0,52.0,29.105000,-7.109847,-0.871811,0.000171,-0.888386,1.0,-2,0
1845964,2018-07-06,KDP,21284300.0,122.919998,124.360001,122.540001,123.949997,2.638189e+09,0.002588,0.021678,...,100.0,0.0,121.572500,0.567949,-0.833820,0.000092,-0.837424,1.0,-2,8
2791136,2020-05-13,SAVA,1996000.0,7.770000,8.820000,7.050000,8.320000,1.660672e+07,-0.086718,0.015873,...,92.0,0.0,7.985000,-16.297786,-0.779793,0.000006,-0.800754,1.0,-2,0
3567481,2023-03-09,WAL,4589800.0,62.360001,70.220001,61.369999,70.180000,3.221122e+08,-0.061263,-0.073042,...,4.0,100.0,70.202499,-2.514235,-0.764351,0.000200,-0.730774,1.0,-2,7
2794015,2024-11-15,SAVE,25732854.0,1.080000,1.340000,1.030000,1.320000,3.396737e+07,0.147826,-0.593846,...,84.0,100.0,2.335000,0.000000,-0.761574,0.000169,-0.767075,1.0,-2,2
